In [ ]:
# -*- coding: utf-8 -*-
import os
import subprocess
import logging
import datetime
import shlex
import json # 主要用於嘗試解析 service-account.json 以驗證，但主要依賴路徑
from google.colab import userdata, drive

# --- 全域設定 ---
REPO_URL = "https://github.com/popcornylu/WolfAI_V2.git" # 應與現有 run_in_colab.ipynb 一致
REPO_PATH = "/content/WolfAI_V2"
BRANCH = "main" # 或其他指定分支
LOG_FILE_PATH = "/content/wolfai_launcher.log"
SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE = "/content/drive/MyDrive/secrets/service-account.json" # 標準路徑

EXPECTED_COLAB_SECRETS = [
    "GOOGLE_API_KEY", # 用於 Gemini
    "API_KEY_FRED",
    "API_KEY_FINMIND",
    "API_KEY_FINNHUB",
    "API_KEY_FMP",
    "ALPHA_VANTAGE_API_KEY",
    "DEEPSEEK_API_KEY",
    "GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT" # 新增，用於 Drive 的持久模式（可選）
]

# --- 日誌設定 ---
def setup_logging():
    """設定日誌記錄器，將日誌同時輸出到控制台和檔案。"""
    try:
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s [%(levelname)s] %(message)s",
            handlers=[
                logging.FileHandler(LOG_FILE_PATH, mode='w', encoding='utf-8'),
                logging.StreamHandler()
            ],
            datefmt="%Y-%m-%d %H:%M:%S"
        )
        logging.info("日誌系統已成功設定。")
    except Exception as e:
        print(f"設定日誌系統時發生錯誤: {e}")

# --- Colab Secrets 讀取 ---
def load_keys_from_colab_secrets():
    """
    從 Google Colab Secrets 中讀取 API 金鑰並設定為環境變數。
    對於 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT，如果存在，會將其內容寫入一個暫存檔案，
    並將 GOOGLE_APPLICATION_CREDENTIALS 環境變數指向該檔案。
    """
    logging.info("正在嘗試從 Colab Secrets 載入 API 金鑰...")
    secrets_loaded_count = 0
    for key_name in EXPECTED_COLAB_SECRETS:
        try:
            value = userdata.get(key_name)
            if value: # 確保值不是空的
                if key_name == "GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT":
                    # 特殊處理服務帳號 JSON 內容
                    temp_sa_path = "/content/temp_service_account.json"
                    with open(temp_sa_path, "w") as f:
                        f.write(value)
                    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = temp_sa_path
                    logging.info(f"✅ 成功從 Colab Secrets 載入並設定服務帳號憑證 (來自 {key_name}) 至 {temp_sa_path}")
                    print(f"✅ 成功從 Colab Secrets 載入並設定服務帳號憑證 (來自 {key_name})")
                else:
                    os.environ[key_name] = value
                    logging.info(f"✅ 成功從 Colab Secrets 載入並設定環境變數: {key_name}")
                    print(f"✅ 成功從 Colab Secrets 載入並設定環境變數: {key_name}")
                secrets_loaded_count += 1
            else:
                logging.warning(f"⚠️ **警告**：在 Colab 密鑰中找到 {key_name}，但其值為空。相關功能可能受限。")
                print(f"⚠️ **警告**：在 Colab 密鑰中找到 {key_name}，但其值為空。相關功能可能受限。")
        except userdata.SecretNotFoundError:
            logging.warning(f"ℹ️提示：在 Colab 密鑰中未找到 {key_name}。您可以稍後在應用程式網頁介面中手動輸入 (如果需要)。")
            print(f"ℹ️提示：在 Colab 密鑰中未找到 {key_name}。您可以稍後在應用程式網頁介面中手動輸入 (如果需要)。")
        except Exception as e:
            logging.error(f"🛑 **錯誤**：讀取 Colab 密鑰 {key_name} 時發生未預期錯誤: {e}")
            print(f"🛑 **錯誤**：讀取 Colab 密鑰 {key_name} 時發生未預期錯誤: {e}")
    
    if secrets_loaded_count > 0:
        logging.info(f"成功從 Colab Secrets 載入 {secrets_loaded_count} 個金鑰/憑證。")
    else:
        logging.warning("未從 Colab Secrets 成功載入任何金鑰或憑證。請確保您已在 Colab 的「密鑰」分頁中設定它們。")
        print("⚠️警告：未從 Colab Secrets 成功載入任何金鑰或憑證。請確保您已在 Colab 的「密鑰」分頁中設定它們。")

# --- 命令執行 ---
def run_command(command_str, cwd=None, shell=False):
    """
    執行給定的命令字串，並即時串流其輸出。
    如果 shell=True，則命令將通過系統的 shell 執行 (應謹慎使用)。
    """
    if not shell:
        command_list = shlex.split(command_str)
        logging.info(f"執行命令 (列表格式): {command_list} (工作目錄: {cwd or os.getcwd()})")
        print(f"\n▶️ 執行命令: {command_str}")
    else:
        command_list = command_str # 給 Popen 傳遞字串
        logging.info(f"執行命令 (Shell): {command_str} (工作目錄: {cwd or os.getcwd()})")
        print(f"\n▶️ 執行命令 (Shell): {command_str}")

    try:
        process = subprocess.Popen(
            command_list,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,  # Line-buffered
            universal_newlines=True,
            cwd=cwd,
            shell=shell # 允許 Popen 使用 shell
        )
        for line in process.stdout:
            print(line, end='') # 即時輸出到 Colab cell
            logging.info(line.strip()) # 記錄到日誌檔案
        process.wait()
        if process.returncode == 0:
            logging.info(f"命令 '{command_str}' 成功執行。")
            print(f"✅ 命令 '{command_str}' 成功執行。")
            return True
        else:
            logging.error(f"命令 '{command_str}' 執行失敗，返回碼: {process.returncode}")
            print(f"🛑 命令 '{command_str}' 執行失敗，返回碼: {process.returncode}")
            return False
    except FileNotFoundError:
        logging.error(f"命令 '{command_list[0] if not shell else command_str}' 未找到。請確保相關程式已安裝且在 PATH 中。")
        print(f"🛑 命令 '{command_list[0] if not shell else command_str}' 未找到。")
        return False
    except Exception as e:
        logging.error(f"執行命令 '{command_str}' 時發生錯誤: {e}", exc_info=True)
        print(f"🛑 執行命令 '{command_str}' 時發生錯誤: {e}")
        return False

# --- 主要工作流程 ---
def main_workflow():
    """執行主要的啟動流程。"""
    logging.info("主工作流程開始...")
    print("\n🚀 正在啟動 WolfAI v5.0 Colab 環境...")

    # 1. 從 Colab Secrets 載入 API 金鑰
    load_keys_from_colab_secrets()

    # 2. 掛載 Google Drive 並處理服務帳號 (如果使用者已提供)
    # 首先檢查 GOOGLE_APPLICATION_CREDENTIALS 是否已由 load_keys_from_colab_secrets 設定
    # (來自 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT Colab Secret)
    use_drive_sa = False
    if os.getenv("GOOGLE_APPLICATION_CREDENTIALS") and os.path.exists(os.getenv("GOOGLE_APPLICATION_CREDENTIALS")):
        logging.info(f"已透過 Colab Secret 'GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT' 設定服務帳號憑證: {os.getenv('GOOGLE_APPLICATION_CREDENTIALS')}")
        print(f"ℹ️ 已透過 Colab Secret 'GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT' 設定服務帳號憑證。")
        try:
            drive.mount('/content/drive', force_remount=True) # 強制重新掛載以確保狀態一致
            logging.info("Google Drive 已成功掛載到 /content/drive (使用 Colab Secret SA)。")
            print("✅ Google Drive 已成功掛載到 /content/drive (使用 Colab Secret SA)。")
            use_drive_sa = True # 表示我們正在使用來自 Colab Secret 的 SA
        except Exception as e:
            logging.error(f"掛載 Google Drive 時發生錯誤 (即使已從 Colab Secret 載入 SA JSON): {e}", exc_info=True)
            print(f"🛑 掛載 Google Drive 時發生錯誤 (即使已從 Colab Secret 載入 SA JSON): {e}")
            print("將嘗試檢查 Drive 中的 'secrets/service-account.json' 路徑...")

    if not use_drive_sa: # 如果沒有透過 Colab Secret 設定 SA，則檢查 Drive 中的固定路徑
        logging.info(f"正在檢查 Google Drive 中的服務帳號檔案: {SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}")
        print(f"\nℹ️ 正在檢查 Google Drive 中的服務帳號檔案: {SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE} (此為可選的進階功能，用於持久模式)")
        try:
            drive.mount('/content/drive', force_remount=True)
            logging.info("Google Drive 已成功掛載到 /content/drive。")
            print("✅ Google Drive 已成功掛載到 /content/drive。")
            if os.path.exists(SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE):
                # 驗證 JSON 檔案內容是否有效 (可選但推薦)
                try:
                    with open(SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE, 'r') as f_sa:
                        json.load(f_sa) # 嘗試解析
                    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE
                    logging.info(f"已設定 GOOGLE_APPLICATION_CREDENTIALS 環境變數為: {SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}")
                    print(f"✅ 服務帳號檔案 '{SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}' 找到並已設定為 GOOGLE_APPLICATION_CREDENTIALS。")
                    use_drive_sa = True
                except json.JSONDecodeError:
                    logging.error(f"🛑 錯誤：位於 '{SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}' 的檔案不是一個有效的 JSON 檔案。請檢查檔案內容。")
                    print(f"🛑 錯誤：位於 '{SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}' 的檔案不是一個有效的 JSON 檔案。")
                except Exception as e_sa_read:
                    logging.error(f"🛑 讀取或解析服務帳號檔案 '{SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}' 時出錯: {e_sa_read}")
                    print(f"🛑 讀取或解析服務帳號檔案 '{SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}' 時出錯: {e_sa_read}")
            else:
                logging.warning(f"未在 '{SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}' 找到服務帳號檔案。如果您打算使用需要 Google Drive 持久儲存的進階功能 (如排程器)，請上傳服務帳號金鑰至該路徑。基礎功能仍可運作。")
                print(f"⚠️ 提示：未在 '{SERVICE_ACCOUNT_JSON_PATH_IN_DRIVE}' 找到服務帳號檔案。基礎功能仍可運作。")
        except Exception as e:
            logging.error(f"掛載 Google Drive 或檢查服務帳號檔案時發生錯誤: {e}", exc_info=True)
            print(f"🛑 掛載 Google Drive 或檢查服務帳號檔案時發生錯誤: {e}")
            print("   這可能是因為您未授權 Colab 存取 Google Drive，或者 Drive API 暫時出現問題。")
            print("   如果您不需要持久模式的進階功能，可以忽略此訊息。")

    # 3. 克隆或更新程式碼倉庫
    logging.info(f"正在處理程式碼倉庫: {REPO_URL}, 分支: {BRANCH}")
    print(f"\n🔄 正在準備程式碼倉庫 (來源: {REPO_URL}, 分支: {BRANCH})...")
    if os.path.exists(REPO_PATH) and os.path.isdir(os.path.join(REPO_PATH, ".git")):
        logging.info(f"倉庫目錄 '{REPO_PATH}' 已存在，嘗試更新...")
        print(f"   倉庫目錄 '{REPO_PATH}' 已存在，嘗試更新...")
        # run_command(f"git fetch origin {BRANCH} && git reset --hard origin/{BRANCH} && git clean -dfx", cwd=REPO_PATH)
        # 使用更安全的更新方式：先 stash 本地改動（如果有的話），再 pull，然後清理
        run_command(f"git stash push -u", cwd=REPO_PATH) # -u 包括未追蹤檔案
        run_command(f"git fetch origin {BRANCH}", cwd=REPO_PATH)
        run_command(f"git checkout {BRANCH}", cwd=REPO_PATH) # 確保在正確的分支
        run_command(f"git reset --hard origin/{BRANCH}", cwd=REPO_PATH)
        run_command("git clean -dfx", cwd=REPO_PATH) # 清理未追蹤檔案和目錄
    else:
        logging.info(f"倉庫目錄 '{REPO_PATH}' 不存在或不是 Git 倉庫，執行克隆...")
        print(f"   倉庫目錄 '{REPO_PATH}' 不存在，執行克隆...")
        if not run_command(f"git clone --branch {BRANCH} {REPO_URL} {REPO_PATH}"):
            logging.critical("克隆倉庫失敗，無法繼續。請檢查 URL、分支名稱和網路連線。")
            print("🛑 克隆倉庫失敗，無法繼續。請檢查 URL、分支名稱和網路連線。")
            return # 嚴重錯誤，終止執行

    # 4. 安裝依賴套件
    requirements_path = os.path.join(REPO_PATH, "backend", "requirements.txt")
    logging.info(f"準備安裝依賴套件: {requirements_path}")
    print(f"\n🐍 正在安裝必要的 Python 套件 (來源: {requirements_path})...")
    if os.path.exists(requirements_path):
        if not run_command(f"pip install --upgrade pip && pip install -r {requirements_path}"):
            logging.critical("安裝依賴套件失敗。請檢查 requirements.txt 檔案和網路連線。")
            print("🛑 安裝依賴套件失敗。")
            # 這裡可以考慮是否要 return，取決於是否希望在依賴安裝失敗後仍嘗試啟動
    else:
        logging.error(f"找不到依賴文件: {requirements_path}。跳過安裝步驟。")
        print(f"🛑 找不到依賴文件: {requirements_path}。跳過安裝步驟。")

    # 5. 啟動應用程式
    start_script_path = os.path.join(REPO_PATH, "scripts", "start.sh")
    logging.info(f"準備啟動應用程式 (腳本: {start_script_path})")
    print(f"\n🚀 正在嘗試啟動 WolfAI 後端服務 (腳本: {start_script_path})...")
    
    if os.path.exists(start_script_path):
        # 確保 start.sh 有執行權限
        run_command(f"chmod +x {start_script_path}")
        
        # 啟動 start.sh。 OPERATION_MODE 現在由後端預設或 .env 檔案控制
        # 我們不再從 Colab 設定 OPERATION_MODE
        # 如果需要，可以在 start.sh 內部或透過 .env 來設定
        print("   後端服務將在背景啟動。請監控下方的日誌輸出。")
        print("   當您看到類似 'Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)' 的訊息時，")
        print("   並且出現一個 '***** Colab Public URL: https://*.trycloudflare.com *****' 的網址時，")
        print("   表示服務已成功啟動。您可以使用該 Public URL 存取 WolfAI 網頁介面。")
        print("   如果長時間未出現 Public URL，請檢查日誌是否有錯誤訊息。")
        
        # 使用 shell=True 來執行 .sh 腳本，因為它可能包含環境變數設定或複雜的 shell 命令
        # 並且 start.sh 預期在 REPO_PATH/scripts 目錄下被 "執行"
        # 但我們希望腳本內的相對路徑是相對於 REPO_PATH/backend
        # start.sh 腳本內部應該自己 cd 到 backend 目錄
        if not run_command(f"{start_script_path}", cwd=os.path.join(REPO_PATH, "scripts"), shell=True):
            logging.error("啟動腳本執行時遇到問題。請查看上面的日誌輸出以獲取詳細資訊。")
            print("🛑 啟動腳本執行時遇到問題。請查看上面的日誌輸出以獲取詳細資訊。")
        else:
            logging.info("啟動腳本已成功執行 (或已在背景開始執行)。")
            print("✅ 啟動腳本已成功執行 (或已在背景開始執行)。")
            print("   請耐心等待幾分鐘，讓服務完全啟動並生成公開網址。")
            print(f"   您可以查看日誌檔案 {LOG_FILE_PATH} 以獲取更詳細的啟動資訊。")

    else:
        logging.critical(f"找不到啟動腳本: {start_script_path}。無法啟動應用程式。")
        print(f"🛑 找不到啟動腳本: {start_script_path}。無法啟動應用程式。")

    logging.info("主工作流程結束。")
    print("\n🎉 WolfAI v5.0 終極啟動器執行完畢！")
    print("   如果一切順利，後端服務正在背景運行。")

# --- 腳本主執行部分 ---
if __name__ == "__main__":
    try:
        setup_logging()
        start_time = datetime.datetime.now()
        logging.info(f"WolfAI v5.0 終極啟動器開始執行於 {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"🚀 WolfAI v5.0 終極啟動器開始執行於 {start_time.strftime('%Y-%m-%d %H:%M:%S')}...")
        print("   日誌將保存在: /content/wolfai_launcher.log")
        print("="*50)

        main_workflow()

        end_time = datetime.datetime.now()
        duration = end_time - start_time
        logging.info(f"WolfAI v5.0 終極啟動器執行完畢於 {end_time.strftime('%Y-%m-%d %H:%M:%S')}。總耗時: {duration}")
        print("="*50)
        print(f"✅ WolfAI v5.0 終極啟動器執行完畢於 {end_time.strftime('%Y-%m-%d %H:%M:%S')}。")
        print(f"   總耗時: {duration}")
        print(f"   請檢查上面的輸出，特別是 Colab Public URL。")

    except Exception as e:
        logging.critical(f"啟動器發生未處理的嚴重錯誤: {e}", exc_info=True)
        print(f"🛑 啟動器發生未處理的嚴重錯誤: {e}")
        print(f"   請檢查日誌檔案 {LOG_FILE_PATH} 以獲取詳細資訊。")
